In [18]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data
from src.models import *
from src.trainer import Trainer
import matplotlib.pyplot as plt
import numpy as np

# Load data

In [19]:
dl_train_all, dl_val_all, dl_test_all = load_all_data(normalize=True)

# Train / Test

## Params

In [22]:
le_net = LeNet()

# Can use any network with ten input and two output nodes here
tail_net = TailLinear()

nb_epochs = 25
n_cv = 1 
n_cv_weight = 1

## Hyperparam opt the weight 

In [23]:
weights = [1e-3, 1e-2, 1e-1, 0.3, 0.5, 0.7, 0.9, 1]

In [24]:
weight_test_accuracies = np.zeros((n_cv_weight, len(weights)))

In [25]:
for j, weight in enumerate(weights): 
    # 5 fold cross validate here 
    for i in range(n_cv_weight):
        siamese_le_net = Siamese(le_net, target=tail_net, weight_aux=weight)
        trainer = Trainer(nb_epochs=nb_epochs)
        trainer.fit(siamese_le_net, dl_train_all, dl_val_all, verbose=False)
        acc_test = trainer.test(siamese_le_net, dl_test_all, test_verbose=True, return_acc=True)
        weight_test_accuracies[i,j] = acc_test

loss_test=0.57	 acc_test=86.23
loss_test=0.81	 acc_test=85.06
loss_test=0.76	 acc_test=85.45
loss_test=0.85	 acc_test=86.43
loss_test=0.74	 acc_test=86.82
loss_test=0.97	 acc_test=88.09
loss_test=1.2	 acc_test=87.5
loss_test=1.43	 acc_test=86.13


In [21]:
avg_weight_test_accuracies = np.mean(weight_test_accuracies, axis=0)

In [24]:
avg_weight_test_accuracies

array([84.102, 86.524, 87.89 , 86.718, 86.582, 87.05 , 86.016, 86.816])

In [26]:
best_weight = weights[np.argmax(avg_weight_test_accuracies)]
best_weight

0.1

In [27]:
# keep best weight

In [28]:
weight_aux = 0.1

In [29]:
test_accuracies = []

In [30]:
for i in range(n_cv):
    siamese_le_net = Siamese(le_net, weight_aux=weight_aux)
    trainer = Trainer(nb_epochs=nb_epochs)
    trainer.fit(siamese_le_net, dl_train_all, dl_val_all, verbose=False)
    acc_test = trainer.test(siamese_le_net, dl_test_all, test_verbose=True, return_acc=True)
    test_accuracies.append(acc_test)

loss_test=4.49	 acc_test=87.11
loss_test=4.11	 acc_test=87.3
loss_test=4.45	 acc_test=87.21
loss_test=5.33	 acc_test=84.96
loss_test=5.56	 acc_test=85.84
loss_test=5.1	 acc_test=85.84
loss_test=4.48	 acc_test=86.72
loss_test=4.54	 acc_test=87.3
loss_test=4.29	 acc_test=86.72
loss_test=4.01	 acc_test=86.52


In [31]:
avg_test_acc = np.mean(test_accuracies)
print('avg test acc', avg_test_acc)
stdev_test_acc = np.std(test_accuracies)
print('stdev test acc', stdev_test_acc)

avg test acc 86.55199999999999
stdev test acc 0.7395376934274542


In [33]:
from src.utils import print_param_count

In [34]:
print_param_count(siamese_le_net)

Total number of parameters:     72268
Number of trainable parameters: 72268
